## 定义基本工具函数

In [1]:
import numpy as np 
import paddle
import paddle.nn.functional as F
import time
import random
from collections import Counter

RANDOM_SEED = 123
paddle.seed(RANDOM_SEED)

def elapsed(sec):
    if sec < 60:
        return str(sec) +"sec"
    elif sec< 60*60:
        return str(sec/60)+"min"
    else :
        return str(sec/(60*60)) +"hr"

training_file = "wordstest.txt" #定义样本文件

def readalltxt(txt_files):
    labels = []
    for txt_file in txt_files:
        target = get_ch_lable(txt_file)
        labels.append(target)

    return training_file

def get_ch_lable(txt_file):
    labels = ""
    with open(txt_file,"rb") as f:
        # print(f) #<_io.BufferedReader name='wordstest.txt'>
        for label in f:
            labels = labels + label.decode("utf-8")

    return labels


def get_ch_lable_v(txt_file,word_num_map,txt_label = None):
    words_size = len(word_num_map)
    to_num = lambda word:word_num_map.get(word,words_size)#如果word没有在训练集出现过，就输出words_size
    if txt_file != None:
        txt_label = get_ch_lable(txt_file)
    labels_vector = list(map(to_num,txt_label))
    return labels_vector

training_data = get_ch_lable(training_file)
print("loaded training data...")

print("样本长度：",len(training_data))

counter  = Counter(training_data)
# print(counter)
words = sorted(counter)
# print(words)
words_size = len(words)
word_num_map = dict(zip(words,range(words_size))) # 字映射到对应的独热编码

print("字表大小",words_size)
wordlabel = get_ch_lable_v(training_file,word_num_map)
# print(wordlabel)

C:\Users\22273\AppData\Roaming\Python\Python311\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


loaded training data...
样本长度： 4240
字表大小 1016


## 模型主体

In [2]:
class GRURNN(paddle.nn.Layer):
    def __init__(self,word_size,embed_dim,hidden_dim,output_size,num_layers):
        super(GRURNN,self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

        self.embed = paddle.nn.Embedding(word_size,embed_dim)
        self.gru = paddle.nn.GRU(input_size=embed_dim,hidden_size=hidden_dim,num_layers=num_layers,direction = "bidirectional")
        self.fc = paddle.nn.Linear(hidden_dim*2,output_size)#输出概率

    def forward(self,features,hidden):
        embedded = self.embed(features.reshape([1,-1]))
        output,hidden = self.gru(embedded.reshape([1,1,-1]),hidden)
        # output = self.attention(output)
        output = self.fc(output.reshape([1,-1]))

        return output,hidden

    def init_zero_state(self):
        init_hidden = paddle.zeros([self.num_layers*2,1,self.hidden_dim])

        return init_hidden


## 训练前的小小准备

In [3]:
EMBEDDING_DIM = 10
HIDDEN_DIM = 20
NUM_LAYERS = 1

model = GRURNN(words_size, EMBEDDING_DIM, HIDDEN_DIM, words_size, NUM_LAYERS)
optimizer = paddle.optimizer.Adam(learning_rate=0.005,parameters=model.parameters())

#定义测试函数
def evaluate(model, prime_str, predict_len, temperature=0.8):

    hidden = model.init_zero_state()
    predicted = ''

    #处理输入语义
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_str[p], hidden)
        predicted +=words[prime_str[p]]
    inp = prime_str[-1]
    predicted +=words[inp]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        #从多项式分布中采样
        output_dist = output.reshape([-1]).divide(paddle.to_tensor(temperature)).exp()
        inp = paddle.multinomial(output_dist, 1)[0]
        
        predicted += words[inp]

    return predicted

## 正式训练

In [4]:

#定义参数训练模型
training_iters = 20000
display_step = 100
n_input = 20
step = 0

offset = random.randint(0,n_input+1)
end_offset = n_input + 1

while step < training_iters:
    start_time = time.time()
    # 随机取一个位置偏移
    if offset > (len(training_data)-end_offset):
        offset = random.randint(0, n_input+1)
   
    inwords =wordlabel[offset:offset+n_input]
    inwords = np.reshape(np.array(inwords), [n_input, -1,  1])

    out_onehot = wordlabel[offset+1:offset+n_input+1]


    hidden = model.init_zero_state()
    # print(hidden)
    optimizer.clear_grad()

    loss = 0.
    inputs, targets = paddle.to_tensor(inwords), paddle.to_tensor(out_onehot)
    for c in range(n_input):
        outputs, hidden = model(inputs[c], hidden)
        loss += F.cross_entropy(outputs, targets[c].reshape([1]))

    loss /= n_input#求每个字的平均loss
    loss.backward()

    optimizer.step()

    #输出日志
    with paddle.set_grad_enabled(False):
        if (step+1) % display_step == 0:
            print(f'Time elapsed: {(time.time() - start_time)/60:.4f} min')
            print(f'step {step+1} | Loss {loss.item():.2f}\n\n')
            with paddle.no_grad():
                print(evaluate(model, inputs, 32), '\n')
            print(50*'=')
            paddle.save(model.state_dict(), 'model_state'+str(2)+'.pdparams')

    step += 1
    offset += (n_input+1)#中间隔了一个，作为预测

print("Finished!")

Time elapsed: 0.0009 min
step 100 | Loss 6.07


个不情愿的挣扎着起床，可是我们还是得用飞时，桑二飘，桥荆那帆事已醉少即的一手中。还的熟果追的季灰涌。即云 

Time elapsed: 0.0009 min
step 200 | Loss 7.00


。朋友是要用关心去润泽，用勉励去雕琢，用而清年的分你，字中任自？，拉在锚张，铃逢丝只一晃游多的得视徘等时 

Time elapsed: 0.0010 min
step 300 | Loss 5.29




　　每一天的早晨我们还是需要坚强，画处，步可找
又道，　　高需光的身豆展又的涌，晚效，迷响，有路棘 

Time elapsed: 0.0010 min
step 400 | Loss 5.84


海遗珠，给予养分，欣赏长处，知己知彼，将的常百开时的秀名，终去幅者当留憧换，些呢碰仇在唯梦的扫候的睁憧了 

Time elapsed: 0.0008 min
step 500 | Loss 4.03


效果，破坏双方的关系。

　　每一天运的　们锚，鹜穿得缺比依，让一破一伤好离从新，请雨们寻连不己，但 

Time elapsed: 0.0011 min
step 600 | Loss 5.89


复。发掘被埋没的那一颗沧海遗珠，给予养分的度己，注享跌慵触虚历能绕一修自程的时光，最谁桥吻的花享己的心复 

Time elapsed: 0.0009 min
step 700 | Loss 5.07


巧，请辅以诚恳温和的态度，否则，忠言逆耳的时成，雁境的时花，习母一一昏泉少，只着　涛如度的笑使，

 

Time elapsed: 0.0009 min
step 800 | Loss 5.58


你步向正途；损友会诱惑你走上歪路，令你陷的锈对，似枕了扣识的时奈，而拨的刹光前，伴的他邃，杯漂受人受舌的 

Time elapsed: 0.0009 min
step 900 | Loss 5.55


面皮薄，请不要在大庭广众上指导，请配合先香的微潺观律是无奈的风雨或壶旧起，十伴着你的歌昏结，职而的间，我 



KeyboardInterrupt: 

## 运行代码

In [ ]:

import paddle
n_input =20 
layer_state_dictm = paddle.load("model_state2.pdparams")
model.set_state_dict(layer_state_dictm)
while True:
    prompt = "请输入几个字，最好是%s个: " % n_input#因为训练我设置的句子长度为20，但是实际输入短一点又无所谓
    sentence = input(prompt)
    if sentence == "break":
        break
    inputword = sentence.strip()
    
    try:
        inputword = get_ch_lable_v(None,word_num_map,inputword)
        keys = np.reshape(np.array(inputword), [ len(inputword),-1, 1])
        zi_num = 400
        
        model.eval()
        with paddle.no_grad():
            sentence =evaluate(model, paddle.to_tensor(keys), zi_num)# zi_num就代表后面继续模型输出字数 总共为n_input + zi_num

        print(sentence)
    # break
    except:
        print("该字我还没学会")


你的梦，激出片伸的心效。

　　　　月我又来的花时，痴事伤，在么界一袭优埃了无微的笑逝，再那定而，静种着陌生。

　　月默般，任终我着的海光，而须鸦的笑，恨份还有婉片，惹，年许年，激茗时，一只的雨的时，蝶抗同的时纯，湿有百下心。

　　　月回美暖的结改，总一最起雨笑恨，句需我绽的声光，给百潮有着的奈，落冰法雨的雨，岁歌思，在头技惹人起往，么你都知者笺，执子的心暖，否诉作有后面，那相笑的迷的薄，有点总的目慷回时，回厚度过美墨Q刻的梦，任在你深彼的起，游的信候，习渐郁，吹泯，那心生时的声，波态会的线纱平而伤的歌目总而在字恍数是我的人个？

　月用有花都押律的时光，试去忧展纯的扬，终新在一长的微笑痴，烦里的荡花们土的暖，悠声郁，让一当往的心，雨是了愿如开的
　　能向伤的嫁格，激涌话到的天，多又如潮极，耳究不声望，否憔挣落过，熟遍出半视的雨边，仿由过的画声光，夕执望是涂场身却的香，滚
该字我还没学会
秋天的风起，耳春微觉阴到，押躇下你的扬，那技磕却适，带的明挣者绊，那滥起眸中的微盛促，雁媚的美。
　　　　千的个而，铃具才往能涛奈，再以醒的微笑小没，有香的一意，恍翩句，不时。一勉毫，执想个心面；承遍古须的放，一份友灰轮，耳普面，不丽，耳以果望的画海却浪，我不文年速，肠睁当不的常，永试的边帜，一千放受坐是种走有着，耳份那改歌等再境，才你凝自份，掬想时深。又张悲心乡，一不迷的情绽，浅人心，我肠苦远的时忆，带如段你多证今你，而精红醒的歌宫经。种着月在我的时由，涛注红局花，成不面行这在什痛场般的河醉，静墨眸，虔不要着你以你优滥的面，请们年。在
　　　　岁月度的眠，飘翩时，一痴慰，回看一然，用渐领而记的信悦，而被趋的首恢声，段在深此之间是命的几路前。

　　月生歌千朵。也站的身缘，滚伤香世，今回这
沐望忆的面弊上的律，低生和来起的究，熟着世旧天起，弹同你。

　　月雨得百线，根在一个不花，我
该字我还没学会
该字我还没学会
该字我还没学会
该字我还没学会



几个例子：
>1. 秋天的风:

>秋天的风茫然，无须多问，无须追究，执子之手，我们注定天长地久。
再不会为了一句话，而和人争得面红耳赤；再不会为了一句话，而和人争得到许，是你的美丽，落在我最暖的心海。而那灰色的头像却让我茫然和无奈，悲由心生，伤从何来？
茫然的心境，遥远的梦幻，飘荡，游离恍Q上成过的人比比皆是。百年仅修得同渡船，要种多少善果才能与他擦肩而过的奔鹜，悲伤的那么？
茫然的心境，遥远的梦幻，飘荡，游离恍若几个世纪的殷虹？也许，是你以来泪。
再不会为了一句话，而和此消魂刻骨。
每一天的早晨我们还是得用飞的圆刻，一行行眼泪的落下的瞬间，终湿透了写满思念的信笺，雁字回时路边，带上我深深地爱恋中，我愿为的是希望大家一起进步。虽然出于一片好的？许许多景，眼泪的渴望。轻轻拉开窗帘，一场春雨，淋湿了虔诚向往的花朵，许许多多的回忆片段如同雨浇开的花香，温馨着那深藏的梦，一如此刻的心境，任那悠悠飘荡的花香

----

>2.风的吹：

>风的吹起，凝眸深处你的微笑惹人痴迷，无奈生命里伏伏起起，你的美永远刻在了最初的单纯，当岁月荒芜了似乎成熟的心田，文字是否仍可以承受很回；石桥上飘荡的魂灵，都舍得了前尘旧梦，断前因后果，忘尽一世浮沉得失，一生爱恨情仇。而我，千年华满天下就人生的方向，从痴痴地心。
　　有时偏执于一己的看法，旁观者清，当局者迷，朋友善意的提点，点出事情的利弊，道出性格的缺陷，为的是希望大家一起进步。虽然出于一片好心，人普遍面皮薄，请配合先赞后弹等技巧，请配合先赞后弹等技巧，请辅以诚恳温和的态度，否则，忠言逆耳，成了反效果，破坏双方的关系。　　
   每一天的早晨我们还是需要坚强，即使远离你无数日子，我依偎在三生石畔夜。
　　再不会为了一句话，而和人争得面红耳赤；每心上的青面拼凑起。夕阳下你的美丽，落日的晚霞像是给你披上漂亮的嫁衣，习习微风把你的面纱吹起，凝眸深处你的微笑惹人痴迷，无奈生命里伏伏起起，你的美

